routerのデータセットでclean labelと noisy labelでどれぐらい判別性能に差が出るのかを検証する(上限と下限づくりということ)

なぜrouter?→ラベルが正しいので

In [1]:
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import rcv1

from cleanlab.noise_generation import generate_noise_matrix, generate_noise_matrix_from_trace, generate_noisy_labels

# visualize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_formats = {'png', 'retina'}

seed=42

### dataの読み込み

In [2]:
data=rcv1.fetch_rcv1()

In [116]:
# データセットがマルチクラス判別なので、シングルクラスにまとめる
mask_col=np.array(list(map(lambda x:x.endswith('CAT'), data.target_names))) #カテゴリーに分けるのが良さそう
target_names=data.target_names[mask_col]
print('target names', target_names) #C→corporate inductrial, E→economics, G→goverment, M→わからない...Market?
mask_row=data.target[:,mask].toarray().sum(axis=1) == 1 #マルチクラスが割り当てられているサンプルは削除

y=data.target[mask_row][:,mask_col]
X=data.data[mask_row]
print('samples',X.shape[0],'category value counts',y.sum(axis=0))
y=np.array(y.argmax(axis=1)).reshape(-1) #one-hot to num

target names ['CCAT' 'ECAT' 'GCAT' 'MCAT']
samples 685071 category value counts [[299612  54695 163135 167629]]


評価に関して、本当はcross validationをするべきなのかもしれない。
しかし今回は簡単のため1つのtestだけで評価する。

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=seed, shuffle=True)

### clean labelで判別

In [120]:
clf=LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_test, y_test) #めっちゃあたる

In [ ]:
clf=RandomForestClassifier(n_estimators=1000, max_depth=10)
clf.fit(X_train,y_train)
clf.score(X_test, y_test)